<a href="https://colab.research.google.com/github/2j0123/WASSUP_EST_FINAL_Team4/blob/SW/MobileNet_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
pip install --upgrade nbformat nbconvert


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.4/257.4 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: mistune
    Found existing installation: mistune 0.8.4
    Uninstalling mistune-0.8.4:
      Successfully uninstalled mistune-0.8.4
  Attempting uninstall: nbconvert
    Found existing installation: nbconvert 6.5.4
    Uninstalling nbconvert-6.5.4:
      Successfully uninstalled nbconvert-6.5.4


In [5]:
!apt-get install git


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.10).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [4]:
!git config --global user.name "Woogie0123"
!git config --global user.email "junircello@gmail.com"

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# 필요한 라이브러리 import
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import os

# 데이터 경로 설정 (훈련 및 검증 데이터)
base_dir = '/content/drive/MyDrive/감정따뜻쟁이/img/train'
validation_dir = '/content/drive/MyDrive/감정따뜻쟁이/img/val'
test_dir = '/content/drive/MyDrive/감정따뜻쟁이/TEST_DATA_SET/TEST_DATA_SET'

# 이미지 데이터 생성기 설정 (훈련 및 검증 데이터)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# 이미지 데이터 로드 (훈련 및 검증 데이터)
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# MobileNetV2 모델 불러오기 (사전 훈련된 가중치 포함)
base_model = MobileNetV2(weights='imagenet', include_top=False)

# 분류기 추가
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)

# 새로운 모델 정의 (기존의 MobileNetV2 모델과 분류기를 결합)
model = Model(inputs=base_model.input, outputs=predictions)

# 모든 레이어를 동결하여 기존의 가중치를 고정
for layer in base_model.layers:
    layer.trainable = False

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 훈련
model.fit(train_generator, epochs=30, validation_data=validation_generator)

# 검증 데이터에 대한 예측
validation_generator.reset()
y_val_pred = model.predict(validation_generator)
y_val_pred_classes = np.argmax(y_val_pred, axis=1)
y_val_true = validation_generator.classes

# 성능 지표 출력 (검증 데이터)
val_target_names = list(validation_generator.class_indices.keys())
print("Validation Data Performance:")
print(classification_report(y_val_true, y_val_pred_classes, target_names=val_target_names))
print(confusion_matrix(y_val_true, y_val_pred_classes))

# 테스트 데이터에 대한 예측
test_generator.reset()
y_test_pred = model.predict(test_generator)
y_test_pred_classes = np.argmax(y_test_pred, axis=1)
y_test_true = test_generator.classes

# 성능 지표 출력 (테스트 데이터)
test_target_names = list(test_generator.class_indices.keys())
print("Test Data Performance:")
print(classification_report(y_test_true, y_test_pred_classes, target_names=test_target_names))
print(confusion_matrix(y_test_true, y_test_pred_classes))

Found 5366 images belonging to 4 classes.
Found 1049 images belonging to 4 classes.
Found 1013 images belonging to 4 classes.


9406464/9406464 [==============================] - 0s 0us/step
Epoch 1/30
168/168 [==============================] - 3166s 19s/step - loss: 1.4563 - accuracy: 0.3377 - val_loss: 1.3762 - val_accuracy: 0.3089
Epoch 2/30
168/168 [==============================] - 869s 5s/step - loss: 1.2646 - accuracy: 0.4117 - val_loss: 1.3311 - val_accuracy: 0.3861
Epoch 3/30
168/168 [==============================] - 832s 5s/step - loss: 1.2401 - accuracy: 0.4419 - val_loss: 1.3820 - val_accuracy: 0.3460
Epoch 4/30
168/168 [==============================] - 866s 5s/step - loss: 1.2317 - accuracy: 0.4385 - val_loss: 1.3290 - val_accuracy: 0.3899
Epoch 5/30
168/168 [==============================] - 851s 5s/step - loss: 1.1895 - accuracy: 0.4607 - val_loss: 1.3200 - val_accuracy: 0.4252
Epoch 6/30
168/168 [==============================] - 827s 5s/step - loss: 1.1827 - accuracy: 0.4734 - val_loss: 1.2976 - val_accuracy: 0.4128
Epoch 7/30
168/168 [==============================] - 834s 5s/step - loss: 1.